### MRO

> MRO — это порядок, в котором Python ищет методы и атрибуты при их вызове. Он особенно важен в контексте множественного наследования, где один класс может наследовать поведение от нескольких других классов. MRO позволяет Python решать, какой метод или атрибут использовать, если они определены в нескольких родительских классах. 

> В Python для вычисления MRO используется алгоритм C3 Linearization (C3-линеаризация).  
>>Два основных правила линеаризации:  
>>__-дети идут раньше родителей;__  
>>__-родители идут в порядке перечисления.__

In [ ]:
def show(_mro):
    c = [f'({v.__name__})' for v in _mro]
    return ' -> '.join(c)

In [ ]:
class A1:
    ...

class B1(A1):
    ...

print(show(B1.__mro__))

(B) -> (A) -> (object)


In [ ]:
class A2:
    ...

class B2(A2):
    ...

class C2(B2):
    ...

print(show(C2.__mro__))

(C2) -> (B2) -> (A2) -> (object)


In [ ]:
# Diamond problem
class A3:
    ...

class B3(A3):
    ...

class C3(A3):
    ...

class D3(B3, C3):
    ...

class E3(C3,  B3):
    ...


print(show(D3.__mro__))
print(show(E3.__mro__))

(D3) -> (B3) -> (C3) -> (A3) -> (object)
(E3) -> (C3) -> (B3) -> (A3) -> (object)


In [ ]:
class A4:
    ...

class B4(A4):
    ...

class C4(A4):
    ...

class D4(B4, C4):
    ...

class E4(C4,  B4):
    ...

class F5(D4, E4):
    ...

print(show(F5.__mro__))

TypeError: Cannot create a consistent method resolution
order (MRO) for bases B4, C4

> Одним из часто используемых механизмов, связанных с MRO, является функция `super()`. Она позволяет обращаться к методам родительских классов, что особенно полезно при переопределении методов в дочерних классах.

In [ ]:
class A6:
    def who_am_i(self):
        print("I am A")
 
class B6(A6):
    def who_am_i(self):
        super().who_am_i()
        print("I am B")
 
class C6(A6):
    def who_am_i(self):
        super().who_am_i()
        print("I am C")
 
class D6(B6, C6):
    def who_am_i(self):
        super().who_am_i()
        print("I am D")
 
d = D6()
d.who_am_i()
print(show(D6.__mro__))

I am A
I am C
I am B
I am D
(D6) -> (B6) -> (C6) -> (A6) -> (object)


> при НЕромбовидной схеме, наследование производится классически (сначала в глубину, затем слева направо) 

In [ ]:
class A7: pass
class B7: pass
class C7(A7): pass
class D7(B7): pass
class E7(C7, D7): pass
print(show(E7.__mro__))

(E7) -> (C7) -> (A7) -> (D7) -> (B7) -> (object)


### Генераторы

> Генератор — это функция, которая возвращает итератор (объект-генератор).  
> Функции-генераторы помогают получать значения по запросу.  
> Объект-генератор можно обойти только один раз.  
> Функции-генераторы экономят память.  
> После возврата всех элементов(прохождения всего цикла), генератор вызовет исключение StopIteration.  
> Инструкция yield возвращает следующее значение и приостанавливает работу функции-генератора.  
> yield можно использовать несколько раз в коде функции-генератора.  
> Помимо yield генератор может содержать и return. Встретив return генератор выбрасывает исключение  StopIteration, а возвращенное значение записывается в объект StopIteration в атрибут value (по умолчанию None)!  

In [ ]:
# пример функции-генератора
def gen(n):
    for i in range(1, n):
        yield i*2

a = gen(10)            # a - объект-генератор !
print(next(a))
print(next(a))

2
4


In [10]:
# пример генераторного выражения
a = (x*2 for x in range(1, 10))
print(next(a))
print(next(a))

2
4


> Есть три метода у генератора:  
> - .close() - остановка вырлднения генератора;  
> - .throw() - поднятие исключения (генератор должен быть инициализирован вызовом next или send(None));  
> - .send() - отпраить значение генератору. Чтобы начать передачу значений в генератор его нужно сначала инициализировать с помощью вызова `next()` или отправив в генератор значение None.

In [11]:
# пример с исключением
def gen_exept(n):
    for i in range(1, 10):
        yield i*2

b = gen_exept(10)
for val in b:
    if val == 6:
        b.throw(Exception('Val is 6!'))
    print(val)

2
4


Exception: Val is 6!